In [4]:
import pandas as pd
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.simplefilter('ignore')
import pyodbc
import sqlite3

In [6]:
servername = 'LAPTOP-RPEMJO16\\SQLEXPRESS'

In [8]:
UO_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=UnitedOutdoors; Trusted_Connection=yes')
odCursor = UO_conn.cursor()
odCursor

In [11]:
order_details = pd.read_sql_query('SELECT * FROM dbo.order_details', UO_conn)
order_details.dtypes


order_details_sk         int64
order_id                 int64
product_quantity         int64
product_listprice      float64
order_date              object
shipping_date           object
product_productid        int64
product_sk              object
employee_employeeid    float64
employee_sk             object
customer_id             object
customer_sk             object
source                  object
dtype: object

In [53]:
order_details['purchaseamount'] = order_details['product_quantity'] * order_details['product_listprice']
order_details
grouped_order_details = order_details.groupby('customer_id')['purchaseamount'].sum().reset_index()
grouped_order_details = grouped_order_details.rename(columns={'purchaseamount': 'totalpurchaseamount'})
grouped_order_details.sort_values('totalpurchaseamount', ascending=False)

order_details_freq = order_details[['customer_id', 'order_date']]
order_details_freq = order_details_freq.drop_duplicates(subset=['customer_id', 'order_date'])
order_details_freq
customer_order_counts = order_details_freq.groupby('customer_id').size().reset_index(name='amountoforders')
customer_order_counts

,customer_id,amountoforders
0,101,11
1,102,8
2,103,8
3,104,11
4,105,6
...,...,...
19312,WARTH,15
19313,WELLI,9
19314,WHITC,14
19315,WILMK,7


In [55]:
amt_purquan = customer_order_counts.merge(grouped_order_details, on='customer_id')
amt_purquan

,customer_id,amountoforders,totalpurchaseamount
0,101,11,5748.00
1,102,8,3624.00
2,103,8,4080.00
3,104,11,6684.00
4,105,6,3540.00
...,...,...,...
19312,WARTH,15,16617.10
19313,WELLI,9,6480.70
19314,WHITC,14,29073.45
19315,WILMK,7,3161.35
